这里其实是“一步”的admm算法

In [1]:
import numpy as np
image_size = 28 # width and length
no_of_different_labels = 10 #  i.e. 0, 1, 2, 3, ..., 9
image_pixels = image_size * image_size
data_path = "data/mnist/"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
                        delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
                       delimiter=",") 

In [2]:
train_data = train_data[:800]
test_data = test_data[:200]

In [3]:
fac = 0.99 / 255
train_imgs = np.asfarray(train_data[:, 1:]) * fac + 0.01
test_imgs = np.asfarray(test_data[:, 1:]) * fac + 0.01

train_labels = np.asfarray(train_data[:, :1])
test_labels = np.asfarray(test_data[:, :1])



lr = np.arange(no_of_different_labels)

# transform labels into one hot representation
train_labels_one_hot = (lr==train_labels).astype(float)
test_labels_one_hot = (lr==test_labels).astype(float)

# we don't want zeroes and ones in the labels neither:
train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99
test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99

In [4]:
import numpy as np
import conv_nn as cnn

准备迭代

In [5]:
k=0
agent_number = 8
#初始化agent
cnn_agent=[]
train_imgs_set=[]
train_labels_one_hot_set = []


for i in range(agent_number):
    cnn_agent.append(cnn.ConvNeuralNetwork(no_of_cnn_kernels=4, 
                            conv_kernel_size=9,
                            image_size=28,
                    no_of_out_nodes=10, 
                    no_of_hidden_nodes=100,
                    learning_rate = 0.1))
    train_imgs_set.append(train_imgs[100*i : 100*(i+1)])
    train_labels_one_hot_set.append(train_labels_one_hot[100*i : 100*(i+1)])
    
#初始化参数z，v_set

z, v = cnn_agent[0].v_z_init_zero()

v_set = []
for i in range(agent_number):
    v_set.append(v)


In [6]:
#增广项的参数
c = 0.001
v_set[i][0].shape
[1,2] + [1,2]
len(train_imgs_set[i])

100

In [7]:
#迭代次数
maxit = 8
#可能是训练的不够，加入epoch

epochs = 10

while(k<maxit):
    #训练agent更新weight
    for i in range(agent_number):
        #每个agent
        for j in range(len(train_imgs_set[i])):
            cnn_agent[i].train_admm(train_imgs_set[i][j], train_labels_one_hot_set[i][j], c, z, v_set[i]) #为了方便

        corrects, wrongs = cnn_agent[i].evaluate(train_imgs, train_labels)
        print("迭代：", k, "agent", i,"accuracy train: ", corrects / ( corrects + wrongs))
        corrects, wrongs = cnn_agent[i].evaluate(test_imgs, test_labels)
        print("迭代：", k, "agent", i,"accuracy: test", corrects / ( corrects + wrongs))

    #更新参数z
    #初始化temp
    temp = []  
    for i in range(len(z)):
        temp.append(np.zeros(z[i].shape))
    for i in range(agent_number):
        temp[0] += (cnn_agent[i].w_kernels + 1.0 / c*v_set[i][0])
        temp[1] += (cnn_agent[i].wih + 1.0 / c*v_set[i][1]) #这个的维数是
        temp[2] += (cnn_agent[i].who + 1.0 / c*v_set[i][2])
    temp[0] = temp[0] / agent_number
    temp[1] = temp[1] / agent_number 
    temp[2] = temp[2] / agent_number 

    
    z = temp 
    
    #c更新v_set
    for i in range(agent_number):
        v_set[i][0] += c*(cnn_agent[i].w_kernels - z[0])
        v_set[i][1] += c*(cnn_agent[i].wih - z[1])
        v_set[i][2] += c*(cnn_agent[i].who - z[2])
    
    k += 1

迭代： 0 agent 0 accuracy train:  0.09125
迭代： 0 agent 0 accuracy: test 0.08
迭代： 0 agent 1 accuracy train:  0.085
迭代： 0 agent 1 accuracy: test 0.065
迭代： 0 agent 2 accuracy train:  0.16875
迭代： 0 agent 2 accuracy: test 0.16
迭代： 0 agent 3 accuracy train:  0.08625
迭代： 0 agent 3 accuracy: test 0.12
迭代： 0 agent 4 accuracy train:  0.08875
迭代： 0 agent 4 accuracy: test 0.115
迭代： 0 agent 5 accuracy train:  0.1625
迭代： 0 agent 5 accuracy: test 0.175
迭代： 0 agent 6 accuracy train:  0.2125
迭代： 0 agent 6 accuracy: test 0.195
迭代： 0 agent 7 accuracy train:  0.09125
迭代： 0 agent 7 accuracy: test 0.1
迭代： 1 agent 0 accuracy train:  0.09125
迭代： 1 agent 0 accuracy: test 0.08
迭代： 1 agent 1 accuracy train:  0.09875
迭代： 1 agent 1 accuracy: test 0.085
迭代： 1 agent 2 accuracy train:  0.12375
迭代： 1 agent 2 accuracy: test 0.145
迭代： 1 agent 3 accuracy train:  0.2225
迭代： 1 agent 3 accuracy: test 0.195
迭代： 1 agent 4 accuracy train:  0.23375
迭代： 1 agent 4 accuracy: test 0.26
迭代： 1 agent 5 accuracy train:  0.26125
迭代： 1 agent

In [8]:
numb = 1
corrects, wrongs = cnn_agent[numb].evaluate(train_imgs, train_labels)
print("accuracy train: ", corrects / ( corrects + wrongs))
corrects, wrongs = cnn_agent[numb].evaluate(test_imgs, test_labels)
print("accuracy: test", corrects / ( corrects + wrongs))

cm = cnn_agent[numb].confusion_matrix(train_imgs, train_labels)
print(cm)

for i in range(10):
    print("digit: ", i, "precision: ", cnn_agent[numb].precision(i, cm), "recall: ", cnn_agent[numb].recall(i, cm))

accuracy train:  0.30625
accuracy: test 0.275
[[ 2  1  1  0  4  1  1  1  0  0]
 [ 7 77 13  6  3  2  1  7  0  6]
 [ 0  0  0  2  0  0  0  0  0  0]
 [54 11 55 58 17 56 21 22 49 35]
 [ 4  8  4  3 28  6  3 24 13  6]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 9  0  6  1  7  5 47  1  8  4]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0]
 [ 3  1  2  3 21  2  0 31  3 33]]
digit:  0 precision:  0.02531645569620253 recall:  0.18181818181818182
digit:  1 precision:  0.7857142857142857 recall:  0.6311475409836066
digit:  2 precision:  0.0 recall:  0.0
digit:  3 precision:  0.7945205479452054 recall:  0.15343915343915343
digit:  4 precision:  0.345679012345679 recall:  0.2828282828282828
digit:  5 precision:  0.0 recall:  nan
digit:  6 precision:  0.6438356164383562 recall:  0.5340909090909091
digit:  7 precision:  0.0 recall:  nan
digit:  8 precision:  0.0 recall:  0.0
digit:  9 precision:  0.39285714285714285 recall:  0.3333333333333333


d:\Program Files\jupyter_file\distribution\conv\conv_nn.py:302: RuntimeWarning: invalid value encountered in long_scalars
  return confusion_matrix[label, label] / row.sum()


In [9]:
i = 1
j = i + 1
print("wih_error: \n",cnn_agent[i].wih - cnn_agent[j].wih)
print("who_error: \n",cnn_agent[i].who - cnn_agent[j].who)
print("w_weights_error: \n",cnn_agent[i].w_kernels[1] - cnn_agent[j].w_kernels[1])


wih_error: 
 [[-0.4704841  -0.11252554  0.38641248 ...  2.82411135  0.15917809
   1.57931041]
 [ 1.47256966  2.38843798 -0.5973372  ...  0.76594199 -0.50440707
  -0.39567832]
 [-1.69556008 -1.99262549  0.15782679 ...  0.89071803  1.03625844
  -0.57944614]
 ...
 [ 0.30845009 -2.25990234 -1.24553114 ... -0.57473722 -0.31419796
  -1.17232614]
 [-0.26904018 -1.30276932  0.07800692 ... -0.39125864  0.60793121
  -0.37199943]
 [ 0.63597855 -0.56912461 -0.14938857 ...  0.17166074 -2.65465861
  -0.03281949]]
who_error: 
 [[-5.93292319e-01 -1.23835948e+00 -1.34376419e+00  1.09365577e+00
  -1.07213634e+00 -9.34950267e-01 -1.35977746e-01 -4.18677486e-01
   2.67624595e-01  9.64589854e-01 -1.15779982e+00  1.66410940e+00
  -1.36702550e+00 -8.52908950e-01  9.69999798e-02  2.72497502e-02
   9.85626363e-02  7.12928528e-01  8.21633838e-01  4.09334515e-01
   6.44592898e-01  1.78907162e+00  6.33367499e-01  2.06214089e+00
  -1.68304445e+00  1.01388754e+00  9.45535599e-01 -1.01451098e+00
   1.25836203e+00  1